In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [ ]:
df = pd.read_json('huanhuan.json')  # 注意修改
ds = Dataset.from_pandas(df)

In [ ]:
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-8B', cache='qwen')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
messages = [
      {"role": "system", "content": "===system_message_test==="},
      {"role": "user", "content": "===user_message_test==="},
      {"role": "assistant", "content": "===assistant_message_test==="},
  ]
text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True,
      enable_thinking=True
  )
print(text)

<|im_start|>system
===system_message_test===<|im_end|>
<|im_start|>user
===user_message_test===<|im_end|>
<|im_start|>assistant
<think>

</think>

===assistant_message_test===<|im_end|>
<|im_start|>assistant



In [ ]:
def process_func(example):
      MAX_LENGTH = 1024  # 设置最大序列长度为1024个token
      input_ids, attention_mask, labels = [], [], []  # 初始化返回值
      # 适配chat_template
      instruction = tokenizer(
          f"<s><|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n"
          f"<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n"
          f"<|im_start|>assistant\n<think>\n\n</think>\n\n",
          add_special_tokens=False
      )
      response = tokenizer(f"{example['output']}", add_special_tokens=False)
      # 将instruction部分和response部分的input_ids拼接，并在末尾添加eos token作为标记结束的token
      input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
      # 注意力掩码，表示模型需要关注的位置
      attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
      # 对于instruction，使用-100表示这些位置不计算loss（即模型不需要预测这部分）
      labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
      if len(input_ids) > MAX_LENGTH:  # 超出最大序列长度截断
          input_ids = input_ids[:MAX_LENGTH]
          attention_mask = attention_mask[:MAX_LENGTH]
          labels = labels[:MAX_LENGTH]
      return {
          "input_ids": input_ids,
          "attention_mask": attention_mask,
          "labels": labels
      }

In [ ]:
print(ds.column_names)
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
print(tokenized_id)

['instruction', 'input', 'output']


Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})


In [ ]:
tokenized_id[0]['input_ids']

[44047,
 29,
 151644,
 8948,
 198,
 99601,
 105182,
 102889,
 104937,
 102144,
 106354,
 313,
 109628,
 123591,
 151645,
 198,
 151644,
 872,
 198,
 102480,
 3837,
 102657,
 100395,
 57750,
 102070,
 30918,
 15946,
 30767,
 3837,
 109042,
 104335,
 102480,
 99172,
 99250,
 121354,
 109453,
 3837,
 107532,
 99623,
 101360,
 88051,
 88051,
 99261,
 9370,
 8545,
 151645,
 198,
 151644,
 77091,
 198,
 151667,
 271,
 151668,
 271,
 119021,
 8545,
 107743,
 99454,
 99513,
 36587,
 99577,
 20412,
 16530,
 99677,
 9370,
 1773,
 151643]

In [ ]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

<s><|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛<|im_end|>
<|im_start|>user
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>
<|im_start|>assistant
<think>

</think>

嘘——都说许愿说破是不灵的。<|endoftext|>


In [ ]:
print(tokenized_id[0]['labels'])

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 119021, 8545, 107743, 99454, 99513, 36587, 99577, 20412, 16530, 99677, 9370, 1773, 151643]


In [ ]:
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"]))))

嘘——都说许愿说破是不灵的。<|endoftext|>


In [ ]:
import torch

model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen3-8B', device_map="auto",torch_dtype=torch.bfloat16)
model

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_la

In [ ]:
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

In [ ]:
model.dtype

torch.bfloat16

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
      task_type=TaskType.CAUSAL_LM,
      target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
      inference_mode=False,  # 训练模式
      r=8,  # Lora 秩
      lora_alpha=32,  # Lora alpha，具体作用参见 Lora 原理
      lora_dropout=0.1  # Dropout 比例
  )
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.1', base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'o_proj', 'down_proj', 'up_proj', 'q_proj', 'k_proj', 'gate_proj', 'v_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)

In [ ]:
model = get_peft_model(model, config)
model

In [ ]:
model.print_trainable_parameters()

In [ ]:
args = TrainingArguments(
      output_dir="./output/Qwen3_8B_lora",
      per_device_train_batch_size=4,
      gradient_accumulation_steps=4,
      logging_steps=10,
      num_train_epochs=3,
      save_steps=300,
      learning_rate=1e-4,
      save_on_each_node=True,
      gradient_checkpointing=True,
      report_to="none",
    #  optim="paged_adamw_8bit",
  )

In [ ]:
trainer = Trainer(
      model=model,
      args=args,
      train_dataset=tokenized_id,
      data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
      # 关键修复点 👇
      # label_names=["labels"]
  )

In [ ]:
trainer.train()

Step,Training Loss
10,4.284146
20,4.070798
30,3.889899
40,4.317472
50,3.878506
60,3.885035
70,3.915145
80,3.917957
90,3.759717
100,3.937287


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=933, training_loss=3.4886013063714945, metrics={'train_runtime': 3583.219, 'train_samples_per_second': 1.041, 'train_steps_per_second': 0.26, 'total_flos': 1.2230982261983232e+16, 'train_loss': 3.4886013063714945, 'epoch': 1.0})

In [ ]:
for batch in trainer.get_train_dataloader():
      print(batch.keys())
      break

KeysView({'input_ids': tensor([[ 44047,     29, 151644,   8948,    198,  99601, 105182, 102889, 104937,
         102144, 106354,    313, 109628, 123591, 151645,    198, 151644,    872,
            198, 112576, 107176, 104258, 113298, 112576,  31235, 100909, 100146,
          56568,   3837,  99654, 104258, 102441, 100245,  87256,  99578,  57452,
          56568,  99369,  17177,   1773, 108595, 109924, 100000,  99693, 109332,
         104591,  99462, 115127,   9370,   3837, 101936, 100000, 100229,  99408,
          44729,  64272,  34187,   3837, 102085,  64471,  20412,  92015,   8863,
          26288,  34187, 115727,   1773, 151645,    198, 151644,  77091,    198,
         151667,    271, 151668,    271,   2073,  56137,  59258,  56137,  99427,
         100413,   3837,  56137,  99194,  56137, 104362,  79766, 104096,   1773,
          56137,  44636,  56137, 111566,   9754,   3837,  56137,  99396,  56137,
         100900, 105013,  32945, 100671,  63703, 104462, 102254, 100698, 123591,
     

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = 'Qwen/Qwen3-8B'
lora_path = './output/Qwen3_8B_lora/checkpoint-933'  # 这里改称你的 Lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16,  trust_remote_code=True)

# 加载Lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)
model

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at './output/Qwen3_8B_lora/checkpoint-933'

In [29]:
prompt = "你是谁？"
inputs = tokenizer.apply_chat_template(
                          [{"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},
                          {"role": "user", "content": prompt}],
                          add_generation_prompt=True,
                          tokenize=True,
                          return_tensors="pt",
                          return_dict=True,
                          enable_thinking=False
                      )
inputs.to(model.device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
      outputs = model.generate(**inputs, **gen_kwargs)
      outputs = outputs[:, inputs['input_ids'].shape[1]:]
      print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

我是甄嬛，家父是大理寺少卿甄远道。


In [ ]:
save_path = './merged_model'  # 合并后模型的保存路径
# 合并LoRA权重到基础模型
merged_model = model.merge_and_unload()

# 保存合并后的模型和tokenizer
merged_model.save_pretrained(save_path, safe_serialization=True)
tokenizer.save_pretrained(save_path)

print(f"模型已成功合并并保存到 {save_path}")
merged_model

In [ ]:
from huggingface_hub import HfApi, login
from google.colab import userdata
# Login (you'll be prompted for your token)
hf_token = userdata.get('HF_TOKEN')
print(hf_token)
login(token=hf_token)

In [ ]:
from huggingface_hub import HfApi, create_repo, login

# Configuration
local_folder = "./merged_model"
repo_id = "KevinXie0131/my_lora_finetuning_huanhuan_0"  # Your repo ID
repo_type = "model"

# Create the repository (with exist_ok=True to avoid error if it exists)
create_repo(
    repo_id=repo_id,
    repo_type=repo_type,
    exist_ok=True,  # This won't error if repo already exists
    private=False,  # Set to True if you want a private repository
)

# Now upload the folder
api = HfApi()
api.upload_folder(
    folder_path=local_folder,
    repo_id=repo_id,
    repo_type=repo_type,
    commit_message="Upload merged Qwen3-8B model from LoRA fine-tuning",
)

print(f"Model successfully uploaded to https://huggingface.co/{repo_id}")

In [ ]:
# from huggingface_hub import HfApi
#
# # Initialize the API
# api = HfApi()
#
# # Upload the entire folder
# api.upload_folder(
#     folder_path="./merged_model",           # Your local folder path
#     repo_id="KevinXie0131/my_lora_finetuning_huanhuan_0", # Your repo ID (e.g., "john/Qwen3-8B-finetuned")
#     repo_type="model",                       # Type of repository: "model", "dataset", or "space"
#     commit_message="Upload merged Qwen3-8B model", # Optional commit message
#     ignore_patterns=["*.tmp", "*.cache"],    # Optional: patterns to ignore
# )